In [1]:
import json
import pandas as pd
import numpy as np 
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import random
from random import choices
from lxml import etree
from xml.dom import minidom

from os import listdir
from os.path import isfile, join

import time

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df_HH = pd.read_csv('data/buildXML2/Appended_HH_data_Lctry.csv')
df_P = pd.read_csv('data/buildXML2/Appended_Person_data_Lctry.csv')
df_flight = pd.read_csv('data/buildXML2/flight_complete.csv')
df_book = pd.read_csv('data/buildXML2/booking_complete.csv')
crosswalk = pd.read_csv('data/geoCrosswalk/GeoCrosswalkMed.csv')
# df_HH_POI = pd.read_csv('data/buildXML//HH_POI.csv')
# df_P_POI = pd.read_csv('data/buildXML/P_POI.csv')
# df_flight = pd.read_csv('data/buildXML/flights_POI.csv')
# df_book_POI = pd.read_csv('data/buildXML/bookings_POI.csv')

/Users/muhammadfathifadlian/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (8,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/muhammadfathifadlian/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_book.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'initID', 'list_of_passengers', 'route',
       'num_in_party', 'BookingAgency', 'BookingDay', 'Unnamed: 0.1.1',
       'IATA_O', 'IATA_D', 'HH_ISO_D', 'region_D', 'HH_ISO_O', 'region_O',
       'LegHH_ISO', 'LegRegion', 'Origin', 'Destination', 'fixRoute',
       'Leg_fixRoute', 'stayDay', 'Airport_Itinerary_Out', 'full_routes',
       'outboundDate', 'returnDate', 'outboundFlights',
       'Airport_Itinerary_Ret', 'return_routes', 'returnFlights'],
      dtype='object')

In [5]:
df_book.groupby('initID')['route'].count().reset_index().sort_values(by='route', ascending=False)

,initID,route
0,CDG-AGP-business-ID0,10
226246,DXB-HAM-leisure-ID29,10
226222,DXB-HAM-leisure-ID268,10
226223,DXB-HAM-leisure-ID269,10
226224,DXB-HAM-leisure-ID27,10
...,...,...
228384,DXB-HEL-leisure-ID1834,1
228383,DXB-HEL-leisure-ID1833,1
228382,DXB-HEL-leisure-ID1832,1
228381,DXB-HEL-leisure-ID1831,1


In [68]:
# df_HH = df_HH.append(df_HH_POI, ignore_index=True)
# df_P = df_P.append(df_P_POI, ignore_index=True)
# # df_flight = df_flight.append(df_flight_POI, ignore_index=True)
# df_book = df_book.append(df_book_POI, ignore_index=True)

In [70]:
# mypath = 'XML_DATAV2'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# suffix_to_remove = '.xml'
 
# # Suffix removal from String list
# # using loop + remove() + endswith()
# new_files = [s.replace(suffix_to_remove, "") for s in onlyfiles]
# if '.DS_Store' in new_files:
#     new_files.remove('.DS_Store')
    
# int_list = [int(item) for item in new_files]

In [6]:
def df_flight_clean(df_flight):
    df_flight['bookings'] = df_flight['bookings'].str.replace("[", "")
    df_flight['bookings'] = df_flight['bookings'].str.replace("]", "")
    df_flight['bookings'] = df_flight['bookings'].str.replace("'", "")
    df_flight['bookings'] = df_flight['bookings'].str.replace(" ", "")
    

    return df_flight

df_flight = df_flight_clean(df_flight)

In [7]:
def df_book_clean(df_book, crosswalk):
    df_book['Airport_Itinerary_Out'] = df_book['Airport_Itinerary_Out'].str.replace("[", "")
    df_book['Airport_Itinerary_Out'] = df_book['Airport_Itinerary_Out'].str.replace("]", "")
    df_book['Airport_Itinerary_Out'] = df_book['Airport_Itinerary_Out'].str.replace("'", "")
    df_book['Airport_Itinerary_Out'] = df_book['Airport_Itinerary_Out'].str.replace(" ", "")
    df_book['Airport_Itinerary_Out'] = df_book['Airport_Itinerary_Out'].str.split(",")
    df_book['IATA'] = df_book['Airport_Itinerary_Out'].str.get(0)
    cw = crosswalk[['IATA', 'Currency', 'HH_ISO', 'City']].copy()
    df_book = df_book.merge(cw, on = 'IATA', how = 'left')
    
    df_book['list_of_passengers'] = df_book['list_of_passengers'].str.replace("[", "")
    df_book['list_of_passengers'] = df_book['list_of_passengers'].str.replace("]", "")
    df_book['list_of_passengers'] = df_book['list_of_passengers'].str.replace("'", "")
    df_book['list_of_passengers'] = df_book['list_of_passengers'].str.replace(" ", "")
    df_book['list_of_passengers'] = df_book['list_of_passengers'].str.split(",")
    df_book['primary_passengers'] = df_book['list_of_passengers'].str.get(0)
    
    df_book['outboundFlights'] = df_book['outboundFlights'].str.replace("[", "")
    df_book['outboundFlights'] = df_book['outboundFlights'].str.replace("]", "")
    df_book['outboundFlights'] = df_book['outboundFlights'].str.replace("'", "")
    df_book['outboundFlights'] = df_book['outboundFlights'].str.replace(" ", "")
    df_book['outboundFlights'] = df_book['outboundFlights'].str.split(",")
    
    df_book['returnFlights'] = df_book['returnFlights'].str.replace("[", "")
    df_book['returnFlights'] = df_book['returnFlights'].str.replace("]", "")
    df_book['returnFlights'] = df_book['returnFlights'].str.replace("'", "")
    df_book['returnFlights'] = df_book['returnFlights'].str.replace(" ", "")
    df_book['returnFlights'] = df_book['returnFlights'].str.split(",")
    
    df_book['allFlights'] = df_book['outboundFlights'] + df_book['returnFlights']
    df_book['allFlights'] = df_book['allFlights'].apply(lambda x: x if isinstance(x, list) else [])
    string_to_remove = 'nan'
    df_book['allFlights'] = df_book['allFlights'].apply(lambda x: [item for item in x if item != string_to_remove])
    df_book['allFlights'] = df_book['allFlights'].apply(lambda x: [int(item) for item in x])
    df_book['allFlights'] = df_book['allFlights'].apply(lambda x: [item for item in x if not pd.isna(item)])
    return df_book
df_book = df_book_clean(df_book, crosswalk)

In [8]:
df_flight = df_flight[df_flight['occupancy'] != 0]
df_flight.groupby('retPNRs')['callsign'].count().reset_index()

,retPNRs,callsign
0,1,25776
1,2,17667
2,3,12514
3,4,9362
4,5,7298
...,...,...
274,288,1
275,291,1
276,292,1
277,295,1


In [9]:
# combined_schedule = df_flight[~df_flight['flight_id'].isin(error_flight_ids)]
print('Number of Flights: ', f"{df_flight['flight_id'].count():,d}")
print('Number of Passengers: ', f"{df_flight['occupancy'].sum().round().astype(int):,d}")
print('Number of Unique Aircraft Callsign: ', f"{df_flight['callsign'].nunique():,d}")
print('Number of Airlines ', df_flight['IATA_A'].nunique())
print('Number of Unique Routes: ', f"{df_flight['LegAirport_IATA'].nunique():,d}")
print('Number of Countries: ', df_flight['HH_ISO_O'].nunique())
print('Number of Cities: ',df_flight['City_O'].nunique())
print('Number of Airports: ', df_flight['IATA_O'].nunique())
print('Number of Recorded Days: ', df_flight['day'].nunique())
print('First Record (day): ', df_flight['day'].min())
print('Last Record (day): ', df_flight['day'].max())

Number of Flights:  172,021
Number of Passengers:  5,649,859
Number of Unique Aircraft Callsign:  5,846
Number of Airlines  161
Number of Unique Routes:  886
Number of Countries:  51
Number of Cities:  135
Number of Airports:  147
Number of Recorded Days:  92
First Record (day):  2019-03-01 00:00:00+00:00
Last Record (day):  2019-05-31 00:00:00+00:00


In [10]:
flight_ids = df_flight['flight_id'].tolist()
# flight_ids = [x for x in flight_ids if x not in int_list]
# len(flight_ids)

In [12]:
flight_ids = flight_ids[0:10]
len(flight_ids)

10

In [18]:
# flight_ids = [141248, 74802, 151717, 153090, 53398, 184710, 185978, 217108]

In [13]:
def preprocess_data(df, index_col):
    """
    Converts a dataframe into a dictionary for faster access.

    Parameters:
    df (pandas.DataFrame): The dataframe to be converted.
    index_col (str): The column name to use as the key in the dictionary.

    Returns:
    dict: A dictionary with keys from the specified index column and values as row data.
    """
    df.fillna('NaN', inplace = True)
    return df.set_index(index_col).T.to_dict()
df_HH_Nat = df_HH[['HHID','NationalityNat']]
df_P = pd.merge(df_P, df_HH_Nat, on='HHID', how = 'left')

flight_data = preprocess_data(df_flight, 'flight_id')
book_data = preprocess_data(df_book, 'initID')
p_data = preprocess_data(df_P, 'P_ID')

/var/folders/j0/bkdgfzfs5bx0c6wgphwnk6kr0000gq/T/ipykernel_26751/872623487.py:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df.set_index(index_col).T.to_dict()


In [14]:
df_P.head()

,Unnamed: 0,P_num,HHID,P_ID,P_AGE,AgeRange,AgeGroup,P_GENDER,GenderHOH,AgeHOH,...,PaymentInfo_VendorCode,PaymentInfo_ExpiryDate,PaymentInfo_AccountNbr,WorkEmail,DOCS_ExpiryDate,DOC_FirstName,DOC_Surname,TYP_FirstName,TYP_Surname,NationalityNat
0,0,1,ALB_0,ALB_0_1,43,40-44,AGE9,F,F,43,...,VISA 19 digit,2022-08-31,4604396803820495590,amyrichardson@tapia.biz,2024-02-14,Amy,Adams,Amy,Adams,ALB
1,1,2,ALB_0,ALB_0_2,86,85-89,AGE18,F,F,43,...,JCB 15 digit,2023-10-17,569834953618,jimenezkeith@jackson.com,2022-01-28,Margaret,Adams,Margaret,Adams,ALB
2,2,3,ALB_1,ALB_1_1,41,40-44,AGE9,M,M,41,...,VISA 19 digit,2023-07-17,4606996421384305,danielgonzalez@carey.com,2023-05-21,Micheal,Wells,Micheal,Wells,ALB
3,3,4,ALB_1,ALB_1_2,41,40-44,AGE9,F,M,41,...,Maestro,2023-11-23,4212297588736775,jennifer19@grant-hill.com,2025-03-29,Anne,Wells,Anne,Wells,ALB
4,4,5,ALB_2,ALB_2_1,62,60-64,AGE13,M,M,62,...,Diners Club / Carte Blanche,2022-11-04,3514986159841637,egeorge@clark-smith.biz,2025-08-13,Timothy,Hogan,Timothy,Hogan,ALB


In [9]:
from joblib import Parallel, delayed
import time, math

In [15]:
def PNRGeneration(i, flight_data, book_data, p_data):
    root = ET.Element("IATA_PNRGOV_NotifRQ")      
    flight_info = flight_data[i]

    a1 = ET.Element("Originator")
    a1.set('AirlineCode', flight_info['IATA_A'])
    a1.set('SystemCode', 'SYSTEMCODE')
    a1.set('AirlineContactInfo', 'AIRLINE CONTACT INFO')
    root.append(a1)
    
    a2 = ET.Element('FlightLeg')
    a2.set('CarrierCode', flight_info['IATA_A'])
    a2.set('FlightNumber', flight_info['flightNbr'])
    a2.set('DepartureDateTime', flight_info['firstseen'])
    a2.set('ArrivalDateTime', flight_info['lastseen'])
    a2.set('DateChangeNbr', str(pd.Timedelta(pd.to_datetime(flight_info['lastseen']) - pd.to_datetime(flight_info['firstseen'])).days))
    
    a2_1 = ET.SubElement(a2, 'DepartureAirport')
    a2_1.set('LocationCode', flight_info['IATA_O'])
    a2_2 = ET.SubElement(a2, 'ArrivalAirport')
    a2_2.set('LocationCode', flight_info['IATA_D'])
    root.append(a2)
    
    a3 = ET.Element('PNRs', NumberOfPnrs = str(flight_info['retPNRs']))
    root.append(a3)

    for j in flight_info['bookings'].split(","):
        print("flight: ", i, " booking :", j)
        if j in book_data:
            booking_info = book_data[j]
            a3_1 = ET.SubElement(a3, 'PNR', NumberOfPassengers = str(booking_info['num_in_party']), 
                                 PNR_TransDate = str(pd.to_datetime(booking_info['outboundDate']) - pd.Timedelta('10 days ')),
                                 PNR_CreationDate = str(pd.to_datetime(booking_info['outboundDate']) - pd.Timedelta('10 days ')),
                                 LastTktDate = str(pd.to_datetime(booking_info['outboundDate']) - pd.Timedelta('10 days ')))
            
            a3_1_1 = ET.SubElement(a3_1, 'BookingRefID', ID=j)
            a3_1_1_1 = ET.SubElement(a3_1_1, 'CompanyName', 
                                     TravelSector = 'TravelSectorCode', 
                                     Code = flight_info['IATA_A'])
                    
            a3_1_2 = ET.SubElement(a3_1, 'POS')
            a3_1_2_1 = ET.SubElement(a3_1_2, 'Source', AgentSine = flight_info['IATA_A'],
                                    PseudoCityCode = booking_info['City'],
                                    ISOCOuntry = booking_info['HH_ISO'],
                                    ISOCurrency = booking_info['Currency'],
                                    AgentDutyCode = "-",
                                    AirlineVendorID = flight_info['IATA_A'],
                                    AirportCode = booking_info['IATA'])
            a3_1_2_1_1 = ET.SubElement(a3_1_2_1, 'RequestorID', Type = str(booking_info['BookingAgency'])[:-1],
                                       ID = "REQUESTOR ID")
            if booking_info['BookingAgency'] == 'Airlines':
                a3_1_2_1_1_1 = ET.SubElement(a3_1_2_1_1, 'CompanyName', Code = flight_info['IATA_A'])
            else:
                a3_1_2_1_1_1 = ET.SubElement(a3_1_2_1_1, 'CompanyName', Code = str(booking_info['BookingAgency']))

            for k in booking_info['list_of_passengers']:
                print("flight: ", i, " booking :", j, " passenger: ", k)
                p_info = p_data[k]
                a3_1_3 = ET.SubElement(a3_1, 'Passenger', RPH=k,
                                        SurnameRefNumber=p_info['HHID'],
                                        BoardingStatus="63")
                a3_1_3_1 = ET.SubElement(a3_1_3, 'GivenName')
                a3_1_3_1.text = p_info['TYP_FirstName']
                a3_1_3_2 = ET.SubElement(a3_1_3, 'Surname')
                a3_1_3_2.text = p_info['TYP_Surname']
                a3_1_3_3 = ET.SubElement(a3_1_3, 'CustLoyalty', 
                                        ProgramID = 'PROGRAM_ID',
                                        MembershipID = j)
                a3_1_3_4 = ET.SubElement(a3_1_3, 'ExcessBaggage', IssuerCode = str(flight_info['IATA_A']),
                                        SerialNumber = k,
                                        SequenceCount = str(1), BaggagePool = "MP", 
                                        UnitOfMeasureQuantity = str(choices([1,2,3], [0.5, 0.3, 0.2])[0]),
                                        UnitOfMeasureCode = "Kg")
                a3_1_3_5 = ET.SubElement(a3_1_3, 'FareInfo', PTC_Code = 'PTC_CODE',
                                        DiscountedFareType = 'DISC_FARE_TYPE',
                                        DiscountrPercent = str(5), CountryCode = str(booking_info['HH_ISO']),
                                        DiscountedFareClassType = 'CABIN_CLASS',
                                        FareBasis = 'SFLY')
                a3_1_3_6 = ET.SubElement(a3_1_3, 'SSR', SSR_Code = 'OTHS',
                                        ServiceQuantity = str(1),Status = 'HN',
                                        BoardPoint = flight_info ['IATA_O'],
                                        OffPoint = flight_info['IATA_D'],
                                        RPH = str(k),
                                        SurnameRefNumber = str(p_info['HHID']))
                a3_1_3_6_1 = ET.SubElement(a3_1_3_6, 'Airline', Code = str(flight_info['IATA_A']))
                a3_1_3_6_2 = ET.SubElement(a3_1_3_6, 'TEXT')
                a3_1_3_6_2.text = 'MADDOX/MOLLY'
                a3_1_3_7 = ET.SubElement(a3_1_3, 'TicketDocument', TicketDocumentNbr = str(i)+"/"+str(j)+"/"+str(k),
                                        Type = 'TICKET_TYPE',
                                        DateOfIssue = str(flight_info['day']),
                                        TicketLocation = flight_info['IATA_O'],
                                        PrimaryDocInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]))
                a3_1_3_7_1 = ET.SubElement(a3_1_3_7, 'TotalFare', 
                                        Amount = str(12345),
                                        CurrencyCode = str(booking_info['Currency']))
                a3_1_3_7_2 = ET.SubElement(a3_1_3_7, 'PricingInfo', Date = str(flight_info['day']),
                                        Time = str(booking_info['outboundDate']),
                                        ISOCountryCode = str(booking_info['HH_ISO']),
                                        LocationCode = str(booking_info['IATA']),
                                        NonEndorsableInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]),
                                        NonRefundableInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]),
                                        PenaltyRestrictionInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]))
                a3_1_3_7_3 = ET.SubElement(a3_1_3_7, 'Taxes')
                a3_1_3_7_3_1 = ET.SubElement(a3_1_3_7_3, 'Tax', Amount = str(123),
                                        ISOCountry = str(booking_info['HH_ISO']),
                                        CurrencyCode = str(booking_info['Currency']),
                                        TaxType = "BK")
                primary_passenger = booking_info['primary_passengers']
                a3_1_3_7_4 = ET.SubElement(a3_1_3_7, 'PaymentInfo', PaymentType = 'CC', PaymentUse = 'NEW',
                                           PaymentAmount = str(12345),
                                           VendorCode = str(p_data[primary_passenger]['PaymentInfo_VendorCode']),
                                           AccountNbr = str(p_data[primary_passenger]['PaymentInfo_AccountNbr']),
                                           ExpiryDate = str(p_data[primary_passenger]['PaymentInfo_ExpiryDate']),
                                       CardHolderName = str(p_data[primary_passenger]['FirstName'])+str(p_data[primary_passenger]['Surname']))
                a3_1_3_8 = ET.SubElement(a3_1_3, 'DOC_SSR', SSR_Code = 'DOCA',
                                     ServiceQuantity = str(1),
                                     Status = 'HK')
                a3_1_3_8_1 = ET.SubElement(a3_1_3_8, 'FlightInfo', Code = str(flight_info['IATA_A']))
                a3_1_3_8_2 = ET.SubElement(a3_1_3_8, 'DOCA', AddressType = "R",
                                      Address = str(p_info['Address']),
                                      CityName = str(booking_info['City']),
                                      PostalCode = str(p_info['PostCode']),
                                      StateProvCounty = 'STATE_PROV_COUNTY',
                                      Country = str(p_info['Country']))
                a3_1_3_9 = ET.SubElement(a3_1_3, 'DOC_SSR', SSR_Code = 'DOCO',ServiceQuantity = str(1),Status = 'HK')
                a3_1_3_9_1 = ET.SubElement(a3_1_3_9, 'DOCO', BirthLocation = str(choices([p_info['Country'], p_info['NationalityNat']], [0.5, 0.5])),
                                       TravelDocType = 'V',
                                       TravelDocNbr = str(k),
                                       PlaceOfIssue = str(p_info['NationalityNat']),
                                       DateOfIssue = "-",
                                       CountryState = str(p_info['NationalityNat']))
                a3_1_3_10 = ET.SubElement(a3_1_3, 'DOC_SSR', SSR_Code = 'DOCS',ServiceQuantity = str(1),Status = 'HK')
                a3_1_3_10_1 = ET.SubElement(a3_1_3_10, 'DOCS', DateOfBirth = str(p_info['DOB']),
                                        ExpiryDate = str(p_info['DOCS_ExpiryDate']),
                                        FirstGivenName = str(p_info['DOC_FirstName']),
                                        SecondGivenName = "",
                                        Surname = str(p_info['DOC_Surname']),
                                        Gender = str(p_info['P_GENDER']),
                                        IssuingLoc = str(p_info['NationalityNat']),
                                        PaxNationality = str(p_info['NationalityNat']))
            fly = 1
            for l in  booking_info['allFlights']:
                flight_list = flight_data[l]
                a3_1_4 = ET.SubElement(a3_1, 'Flight', 
                DepartureDateTime= str(flight_list['firstseen']),
                ArrivalDateTime = str(flight_list['lastseen']),
                ResBookDesigCode = "S",
                NumberInParty = str(booking_info['num_in_party']),
                Status = "HK",
                FlightNumber = str(fly))
                a3_1_4_1 = ET.SubElement(a3_1_4, 'DepartureAirport', LocationCode = str(flight_list['IATA_O']))
                a3_1_4_2 = ET.SubElement(a3_1_4, 'ArrivalAirport', LocationCode = str(flight_list['IATA_D']))
                a3_1_4_3 = ET.SubElement(a3_1_4, 'OperatingAirline', Code = str(flight_list['IATA_A']),
                                         FlightNumber = str(flight_list['flightNbr']),
                                         ResBookDesigCode = "S")
                a3_1_4_4 = ET.SubElement(a3_1_4, 'MarketingAirline', Code = "TX")
            
                fly = fly+1

    tree = ET.ElementTree(root)    
    filename = 'data/XML_Test/'+str(i)+".xml"        
      
    with open (filename, "wb") as file :
        file.write(ET.tostring(root,encoding='UTF-8'))

In [28]:
# def PNRGeneration_old(i):
    
#     root = ET.Element("IATA_PNRGOV_NotifRQ")      
#     a1 = ET.Element("Originator")
#     a1.set('AirlineCode', str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]))
#     a1.set('SystemCode', 'SYSTEMCODE')
#     a1.set('AirlineContactInfo', 'AIRLINE CONTACT INFO')
#     root.append(a1)
    
#     a2 = ET.Element('FlightLeg')
#     a2.set('CarrierCode', str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]))
#     a2.set('FlightNumber', str(df_flight.loc[df_flight['flight_id'] == i, 'flightNbr'].iloc[0]))
#     a2.set('DepartureDateTime', str(df_flight.loc[df_flight['flight_id'] == i, 'firstseen'].iloc[0]))
#     a2.set('ArrivalDateTime', str(df_flight.loc[df_flight['flight_id'] == i, 'lastseen'].iloc[0]))
#     a2.set('DateChangeNbr', str(pd.Timedelta(pd.to_datetime(df_flight.loc[df_flight['flight_id'] == i, 'lastseen'].iloc[0]) - pd.to_datetime(df_flight.loc[df_flight['flight_id'] == i, 'firstseen'].iloc[0])).days))
    
#     a2_1 = ET.SubElement(a2, 'DepartureAirport')
#     a2_1.set('LocationCode', df_flight.loc[df_flight['flight_id'] == i, 'IATA_O'].iloc[0])
#     a2_2 = ET.SubElement(a2, 'ArrivalAirport')
#     a2_2.set('LocationCode', df_flight.loc[df_flight['flight_id'] == i, 'IATA_D'].iloc[0])
#     root.append(a2)
    
#     a3 = ET.Element('PNRs', NumberOfPnrs = str(df_flight.loc[df_flight['flight_id'] == i, 'retPNRs'].iloc[0]))
#     root.append(a3)
    
# #     print(i)
#     for j in df_flight.loc[df_flight['flight_id'] == i, 'bookings'].iloc[0].split(","):
# #         print(j)
#         a3_1 = ET.SubElement(a3, 'PNR', NumberOfPassengers = str(df_book.loc[df_book['initID'] == j, 'num_in_party'].iloc[0]), 
#                              PNR_TransDate = str(pd.to_datetime(df_book.loc[df_book['initID'] == j, 'outboundDate'].iloc[0]) - pd.Timedelta('10 days ')),
#                              PNR_CreationDate = str(pd.to_datetime(df_book.loc[df_book['initID'] == j, 'outboundDate'].iloc[0]) - pd.Timedelta('10 days ')),
#                              LastTktDate = str(pd.to_datetime(df_book.loc[df_book['initID'] == j, 'outboundDate'].iloc[0]) - pd.Timedelta('10 days ')))
        
#         a3_1_1 = ET.SubElement(a3_1, 'BookingRefID', ID=str(j))
#         a3_1_1_1 = ET.SubElement(a3_1_1, 'CompanyName', 
#                                  TravelSector = 'TravelSectorCode', 
#                                  Code = str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]))
                
#         a3_1_2 = ET.SubElement(a3_1, 'POS')
#         a3_1_2_1 = ET.SubElement(a3_1_2, 'Source', AgentSine = str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]),
#                                 PseudoCityCode = str(df_book.loc[df_book['initID'] == j, 'City'].iloc[0]),
#                                 ISOCOuntry = str(df_book.loc[df_book['initID'] == j, 'HH_ISO'].iloc[0]),
#                                 ISOCurrency = str(df_book.loc[df_book['initID'] == j, 'Currency'].iloc[0]),
#                                 AgentDutyCode = "-",
#                                 AirlineVendorID = str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]),
#                                 AirportCode = str(df_book.loc[df_book['initID'] == j, 'IATA'].iloc[0]))
#         a3_1_2_1_1 = ET.SubElement(a3_1_2_1, 'RequestorID', ID = "REQUESTOR ID")
#         a3_1_2_1_1_1 = ET.SubElement(a3_1_2_1_1, 'CompanyName', Code = str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]))
        
#         for k in df_book.loc[df_book['initID'] == j, 'list_of_passengers'].iloc[0]:
# #             print(k)
#             a3_1_3 = ET.SubElement(a3_1, 'Passenger', RPH = k,
#                                    SurnameRefNumber = str(df_P.loc[df_P['P_ID'] == k, 'HHID'].iloc[0]),
#                                    BoardingStatus = "63")
#             a3_1_3_1 = ET.SubElement(a3_1_3, 'GivenName')
#             a3_1_3_1.text = str(df_P.loc[df_P['P_ID'] == k, 'FirstName'].iloc[0])
#             a3_1_3_2 = ET.SubElement(a3_1_3, 'Surname')
#             a3_1_3_2.text = str(df_P.loc[df_P['P_ID'] == k, 'Surname'].iloc[0])
#             a3_1_3_3 = ET.SubElement(a3_1_3, 'CustLoyalty', 
#                                      ProgramID = 'PROGRAM_ID',
#                                      MembershipID = j)
#             a3_1_3_4 = ET.SubElement(a3_1_3, 'ExcessBaggage', IssuerCode = str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]),
#                                      SerialNumber = k,
#                                      SequenceCount = str(1), BaggagePool = "MP", 
#                                      UnitOfMeasureQuantity = str(choices([1,2,3], [0.5, 0.3, 0.2])[0]),
#                                      UnitOfMeasureCode = "Kg")
#             a3_1_3_5 = ET.SubElement(a3_1_3, 'FareInfo', PTC_Code = 'PTC_CODE',
#                                      DiscountedFareType = 'DISC_FARE_TYPE',
#                                      DiscountrPercent = str(5), CountryCode = str(df_book.loc[df_book['initID'] == j, 'HH_ISO'].iloc[0]),
#                                      DiscountedFareClassType = 'CABIN_CLASS',
#                                      FareBasis = 'SFLY')
#             a3_1_3_6 = ET.SubElement(a3_1_3, 'SSR', SSR_Code = 'OTHS',
#                                      ServiceQuantity = str(1),Status = 'HN',
#                                      BoardPoint = df_flight.loc[df_flight['flight_id'] == i, 'IATA_O'].iloc[0],
#                                      OffPoint = df_flight.loc[df_flight['flight_id'] == i, 'IATA_D'].iloc[0],
#                                      RPH = str(k),
#                                      SurnameRefNumber = str(df_P.loc[df_P['P_ID'] == k, 'HHID'].iloc[0]))
#             a3_1_3_6_1 = ET.SubElement(a3_1_3_6, 'Airline', Code = str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]))
#             a3_1_3_6_2 = ET.SubElement(a3_1_3_6, 'TEXT')
#             a3_1_3_6_2.text = 'MADDOX/MOLLY'
#             a3_1_3_7 = ET.SubElement(a3_1_3, 'TicketDocument', TicketDocumentNbr = str(i)+"/"+str(j)+"/"+str(k),
#                                      Type = 'TICKET_TYPE',
#                                      DateOfIssue = str(df_flight.loc[df_flight['flight_id'] == i, 'day'].iloc[0]),
#                                      TicketLocation = df_flight.loc[df_flight['flight_id'] == i, 'IATA_O'].iloc[0],
#                                      PrimaryDocInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]))
#             a3_1_3_7_1 = ET.SubElement(a3_1_3_7, 'TotalFare', 
#                                        Amount = str(12345),
#                                        CurrencyCode = str(df_book.loc[df_book['initID'] == j, 'Currency'].iloc[0]))
#             a3_1_3_7_2 = ET.SubElement(a3_1_3_7, 'PricingInfo', Date = str(df_flight.loc[df_flight['flight_id'] == i, 'day'].iloc[0]),
#                                        Time = str(df_book.loc[df_book['initID'] == j, 'outboundDate'].iloc[0]),
#                                        ISOCountryCode = str(df_book.loc[df_book['initID'] == j, 'HH_ISO'].iloc[0]),
#                                        LocationCode = str(df_book.loc[df_book['initID'] == j, 'IATA'].iloc[0]),
#                                        NonEndorsableInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]),
#                                        NonRefundableInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]),
#                                        PenaltyRestrictionInd = str(choices(['TRUE', 'FALSE'], [0.7, 0.3])[0]))
#             a3_1_3_7_3 = ET.SubElement(a3_1_3_7, 'Taxes')
#             a3_1_3_7_3_1 = ET.SubElement(a3_1_3_7_3, 'Tax', Amount = str(123),
#                                          ISOCountry = str(df_book.loc[df_book['initID'] == j, 'HH_ISO'].iloc[0]),
#                                          CurrencyCode = str(df_book.loc[df_book['initID'] == j, 'Currency'].iloc[0]),
#                                          TaxType = "BK")
#             primary_passenger = df_book.loc[df_book['initID'] == j, 'primary_passengers'].iloc[0]
#             a3_1_3_7_4 = ET.SubElement(a3_1_3_7, 'PaymentInfo', PaymentType = 'CC', PaymentUse = 'NEW',
#                                        PaymentAmount = str(12345),
#                                        VendorCode = str(df_P.loc[df_P['P_ID'] == primary_passenger, 'PaymentInfo_VendorCode'].iloc[0]),
#                                        AccountNbr = str(df_P.loc[df_P['P_ID'] == primary_passenger, 'PaymentInfo_AccountNbr'].iloc[0]),
#                                        ExpiryDate = str(df_P.loc[df_P['P_ID'] == primary_passenger, 'PaymentInfo_ExpiryDate'].iloc[0]),
#                                        CardHolderName = str(df_P.loc[df_P['P_ID'] == primary_passenger, 'FirstName'].iloc[0])+str(df_P.loc[df_P['P_ID'] == primary_passenger, 'Surname'].iloc[0]))
#             a3_1_3_8 = ET.SubElement(a3_1_3, 'DOC_SSR', SSR_Code = 'DOCA',
#                                      ServiceQuantity = str(1),
#                                      Status = 'HK')
#             a3_1_3_8_1 = ET.SubElement(a3_1_3_8, 'FlightInfo', Code = str(df_flight.loc[df_flight['flight_id'] == i, 'IATA_A'].iloc[0]))
#             a3_1_3_8_2 = ET.SubElement(a3_1_3_8, 'DOCA', AddressType = "R",
#                                       Address = str(df_P.loc[df_P['P_ID'] == k, 'Address'].iloc[0]),
#                                       CityName = str(df_book.loc[df_book['initID'] == j, 'City'].iloc[0]),
#                                       PostalCode = str(df_P.loc[df_P['P_ID'] == k, 'PostCode'].iloc[0]),
#                                       StateProvCounty = 'STATE_PROV_COUNTY',
#                                       Country = str(df_P.loc[df_P['P_ID'] == k, 'Country'].iloc[0]))
#             a3_1_3_9 = ET.SubElement(a3_1_3, 'DOC_SSR', SSR_Code = 'DOCO',ServiceQuantity = str(1),Status = 'HK')
#             a3_1_3_9_1 = ET.SubElement(a3_1_3_9, 'DOCO', BirthLocation = str(df_P.loc[df_P['P_ID'] == k, 'Country'].iloc[0]),
#                                        TravelDocType = 'V',
#                                        TravelDocNbr = str(k),
#                                        PlaceOfIssue = str(df_P.loc[df_P['P_ID'] == k, 'Country'].iloc[0]),
#                                        DateOfIssue = "-",
#                                        CountryState = str(df_P.loc[df_P['P_ID'] == k, 'Country'].iloc[0]))
#             a3_1_3_10 = ET.SubElement(a3_1_3, 'DOC_SSR', SSR_Code = 'DOCS',ServiceQuantity = str(1),Status = 'HK')
#             a3_1_3_10_1 = ET.SubElement(a3_1_3_10, 'DOCS', DateOfBirth = str(df_P.loc[df_P['P_ID'] == k, 'DOB'].iloc[0]),
#                                         ExpiryDate = str(df_P.loc[df_P['P_ID'] == k, 'DOCS_ExpiryDate'].iloc[0]),
#                                         FirstGivenName = str(df_P.loc[df_P['P_ID'] == k, 'FirstName'].iloc[0]),
#                                         SecondGivenName = "",
#                                         Surname = str(df_P.loc[df_P['P_ID'] == k, 'Surname'].iloc[0]),
#                                         Gender = str(df_P.loc[df_P['P_ID'] == k, 'P_GENDER'].iloc[0]),
#                                         IssuingLoc = str(df_P.loc[df_P['P_ID'] == k, 'Country'].iloc[0]),
#                                         PaxNationality = str(df_P.loc[df_P['P_ID'] == k, 'Country'].iloc[0]))
#         fly = 1
# #         print(j)
# #         print(len(df_book.loc[df_book['initID'] == j, 'allFlights'].iloc[0]))
#         for l in  df_book.loc[df_book['initID'] == j, 'allFlights'].iloc[0]:
            
#             a3_1_4 = ET.SubElement(a3_1, 'Flight', 
#                                    DepartureDateTime= str(df_flight.loc[df_flight['flight_id'] == l, 'firstseen'].iloc[0]),
#                                    ArrivalDateTime = str(df_flight.loc[df_flight['flight_id'] == l, 'lastseen'].iloc[0]),
#                                    ResBookDesigCode = "S",
#                                    NumberInParty = str(df_book.loc[df_book['initID'] == j, 'num_in_party'].iloc[0]),
#                                    Status = "HK",
#                                    FlightNumber = str(fly))
#             a3_1_4_1 = ET.SubElement(a3_1_4, 'DepartureAirport', LocationCode = str(df_flight.loc[df_flight['flight_id'] == l, 'IATA_O'].iloc[0]))
#             a3_1_4_2 = ET.SubElement(a3_1_4, 'ArrivalAirport', LocationCode = str(df_flight.loc[df_flight['flight_id'] == l, 'IATA_D'].iloc[0]))
#             a3_1_4_3 = ET.SubElement(a3_1_4, 'OperatingAirline', Code = str(df_flight.loc[df_flight['flight_id'] == l, 'IATA_A'].iloc[0]),
#                                     FlightNumber = str(df_flight.loc[df_flight['flight_id'] == l, 'flightNbr'].iloc[0]),
#                                     ResBookDesigCode = "S")
#             a3_1_4_4 = ET.SubElement(a3_1_4, 'MarketingAirline', Code = "TX")
            
#             fly = fly+1
            
    
    
#     tree = ET.ElementTree(root)    
#     filename = 'data/XML_DATAV3'+str(i)+".xml"        
      
#     with open (filename, "wb") as file :
#         file.write(ET.tostring(root,encoding='UTF-8'))

In [19]:
Parallel(n_jobs=6)(delayed(PNRGeneration)(i, flight_data, book_data, p_data) for i in flight_ids)

NameError: name 'Parallel' is not defined

In [16]:
# List to store flight_ids that caused errors
error_flight_ids = []

# Start the timer
start_time = time.time()

# Run your function for each item in the list
for i in flight_ids:
    try:
        # print(i)
        PNRGeneration(i, flight_data, book_data, p_data)
    except Exception as e:
        print(f"Error processing flight_id {i}: {e}")
        error_flight_ids.append(i)
        # Optionally, log more information or handle the exception in other ways

# Calculate the total time taken
duration_without_parallel = time.time() - start_time
print(f"Duration without parallel processing: {duration_without_parallel} seconds")

# Print or handle error_flight_ids as needed
print("Flight IDs with errors:", error_flight_ids)



flight:  56  booking : CDG-LAX-group-ID3
flight:  56  booking : CDG-LAX-group-ID3  passenger:  BIH_49540_1
flight:  56  booking : CDG-LAX-leisure-ID13
flight:  56  booking : CDG-LAX-leisure-ID13  passenger:  GGY_35218_1
flight:  73  booking : LHR-LAX-leisure-ID3
flight:  73  booking : LHR-LAX-leisure-ID3  passenger:  GGY_20665_1
flight:  73  booking : LHR-LAX-leisure-ID3  passenger:  GGY_20665_2
flight:  73  booking : LHR-LAX-leisure-ID29
flight:  73  booking : LHR-LAX-leisure-ID29  passenger:  JEY_20796_1
flight:  73  booking : LHR-LAX-leisure-ID29  passenger:  JEY_20796_2
flight:  73  booking : LHR-LAX-leisure-ID47
flight:  73  booking : LHR-LAX-leisure-ID47  passenger:  MKD_26383_1
flight:  82  booking : LHR-SAN-leisure-ID2
flight:  82  booking : LHR-SAN-leisure-ID2  passenger:  IMN_28854_1
flight:  91  booking : CDG-EZE-leisure-ID75
flight:  91  booking : CDG-EZE-leisure-ID75  passenger:  POL_19240_1
flight:  91  booking : CDG-EZE-leisure-ID75  passenger:  POL_19240_2
flight:  118 

In [9]:
df_book[df_book['initID'] == 'CDG-AGP-business-ID0']

,Unnamed: 0,Unnamed: 0.1,initID,list_of_passengers,route,num_in_party,BookingAgency,BookingDay,Unnamed: 0.1.1,IATA_O,...,Leg_fixRoute,stayDay,Airport_Itinerary_Out,full_routes,outboundDate,returnDate,outboundFlights,Airport_Itinerary_Ret,return_routes,returnFlights
22288,22288,22288,CDG-AGP-business-ID0,"['ROU_5149_1', 'GRC_5018_1', 'GRC_1905_1', 'GR...",CDG-AGP,4,BookingAgency1,31,1130.0,CDG,...,1,16.0,"['CDG', 'AGP']",['CDG-AGP'],2019-03-26 00:00:00+00:00,2019-04-11 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[886612]
193045,193045,19500,CDG-AGP-business-ID0,"['SWE_2895_1', 'POL_3809_1', 'POL_1935_1', 'MK...",CDG-AGP,4,BookingAgency2,43,1130.0,CDG,...,1,3.0,"['CDG', 'AGP']",['CDG-AGP'],2019-04-03 00:00:00+00:00,2019-04-06 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[827211]
369116,369116,47751,CDG-AGP-business-ID0,"['TZA_62687_1', 'TZA_56921_1', 'TZA_47931_1', ...",CDG-AGP,4,BookingAgency3,16,1130.0,CDG,...,1,10.0,"['CDG', 'AGP']",['CDG-AGP'],2019-04-19 00:00:00+00:00,2019-04-29 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[1109593]
759905,759905,10356,CDG-AGP-business-ID0,"['ZAF_6610_1', 'ZAF_2383_1', 'SSD_3791_1', 'SD...",CDG-AGP,4,BookingAgency2,36,1130.0,CDG,...,1,7.0,"['CDG', 'AGP']",['CDG-AGP'],2019-03-27 00:00:00+00:00,2019-04-03 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[787221]
864945,864945,12099,CDG-AGP-business-ID0,['USA_11858_1'],CDG-AGP,1,Airlines,52,1130.0,CDG,...,1,2.0,"['CDG', 'AGP']",['CDG-AGP'],2019-03-08 00:00:00+00:00,2019-03-10 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[519522]
968438,968438,12323,CDG-AGP-business-ID0,"['VEN_9182_1', 'PER_635_1', 'PAN_2701_1', 'PAN...",CDG-AGP,4,Airlines,41,1130.0,CDG,...,1,11.0,"['CDG', 'AGP']",['CDG-AGP'],2019-03-12 00:00:00+00:00,2019-03-23 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[666957]
1073531,1073531,13842,CDG-AGP-business-ID0,"['TTO_9598_1', 'SUR_7806_1']",CDG-AGP,2,Airlines,38,1130.0,CDG,...,1,5.0,"['CDG', 'AGP']",['CDG-AGP'],2019-04-12 00:00:00+00:00,2019-04-17 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[967527]
1171469,1171469,8124,CDG-AGP-business-ID0,"['QAT_2732_1', 'PHL_3950_1', 'PHL_3054_1']",CDG-AGP,3,BookingAgency1,35,1130.0,CDG,...,1,NaN,"['CDG', 'AGP']",['CDG-AGP'],2019-04-07 00:00:00+00:00,NaN,[nan],"['AGP', 'CDG']",['AGP-CDG'],[nan]
1274262,1274262,12455,CDG-AGP-business-ID0,['UZB_2804_1'],CDG-AGP,1,BookingAgency2,24,1130.0,CDG,...,1,13.0,"['CDG', 'AGP']",['CDG-AGP'],2019-05-11 00:00:00+00:00,2019-05-24 00:00:00+00:00,[nan],"['AGP', 'CDG']",['AGP-CDG'],[302741]
1370825,1370825,10906,CDG-AGP-business-ID0,"['YEM_2768_1', 'VNM_12663_1', 'VNM_10182_1', '...",CDG-AGP,4,BookingAgency2,48,1130.0,CDG,...,1,NaN,"['CDG', 'AGP']",['CDG-AGP'],2019-05-27 00:00:00+00:00,NaN,[nan],"['AGP', 'CDG']",['AGP-CDG'],[nan]


In [15]:
df_flight[df_flight['flight_id'] == 518]

,Unnamed: 0,flight_id,callsign,number,icao24,registration,typecode,origin,destination,firstseen,...,Country_A,capacity,routeOperator,origin_outlier,destination_outlier,LegAirport_ICAO_outlier,occupancy,bookings,outPNRs,retPNRs
169,169,518,DLH631,NaN,3c6564,D-AIKD,A333,OMDB,EDDF,2019-04-30 21:10:04+00:00,...,Germany,335.0,OMDB-EDDF-DLH,Normal,Normal,Normal,230,"DXB-FRA-business-ID132,DXB-GOT-business-ID12,D...",104,104


In [35]:
len(error_flight_ids)

245

In [143]:
error_flight_ids_old = []

# Start the timer
start_time = time.time()

# Run your function for each item in the list
for i in flight_ids[0:100]:
    try:
        print(i)
        PNRGeneration_old(i)
    except Exception as e:
        print(f"Error processing flight_id {i}: {e}")
        error_flight_ids_old.append(i)
        # Optionally, log more information or handle the exception in other ways

# Calculate the total time taken
duration_without_parallel = time.time() - start_time
print(f"Duration without parallel processing and Old Algorithm: {duration_without_parallel} seconds")

# Print or handle error_flight_ids as needed
print("Flight IDs with errors:", error_flight_ids_old)

25
49
53
57
65
72
73
77
89
100
117
118
119
128
138
141
147
150
159
163
164
166
173
183
210
248
254
265
295
307
315
342
403
454
493
494
506
532
558
562
606
662
725
768
797
805
823
839
869
876
882
883
886
908
920
932
939
968
969
989
1000
1001
1004
1012
1014
1023
1044
1056
1059
1067
1068
1069
1074
1080
1090
1095
1111
1112
1113
1120
1121
1122
1124
1129
1131
1137
1143
1144
1146
1147
1155
1165
1173
1174
1188
1191
1196
1199
1201
1210
Duration without parallel processing and Old Algorithm: 55622.18280315399 seconds
Flight IDs with errors: []


In [86]:
start_time = time.time()

# Run your function in parallel
Parallel(n_jobs=6)(delayed(PNRGeneration)(i, flight_data, book_data, p_data) for i in flight_ids[0:100])

# Calculate the total time taken
duration_with_parallel = time.time() - start_time
print(f"Duration with parallel processing: {duration_with_parallel} seconds")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


KeyboardInterrupt: 

In [60]:
df_flight

,Unnamed: 0,Unnamed: 0.1,flight_id,callsign,number,icao24,registration,typecode,origin,destination,...,Country_A,capacity,routeOperator,origin_outlier,destination_outlier,LegAirport_ICAO_outlier,occupancy,bookings,outPNRs,retPNRs
0,0,0,0,CSN461,NaN,780aec,B-2042,B77L,KORD,EDDF,...,China,320.0,KORD-EDDF-CSN,Normal,Normal,Normal,0,,1,1
1,1,1,2,CCA855,CA855,7804ca,B-6505,A332,OMDB,EGLL,...,China,293.0,OMDB-EGLL-CCA,Normal,Normal,Normal,0,,1,1
2,2,2,3,AAR501,NaN,71c254,HL8254,B772,KORD,LFPG,...,South Korea,312.0,KORD-LFPG-AAR,Normal,Normal,Normal,0,,1,1
3,3,3,10,THA926,NaN,885179,HS-TKY,B77W,EKCH,EDDF,...,Thailand,212.0,EKCH-EDDF-THA,Normal,Normal,Normal,0,,1,1
4,4,4,20,AFR293,AF293,394a09,F-GSQJ,B77W,RJTT,LFPG,...,France,212.0,RJTT-LFPG-AFR,Normal,Normal,Normal,0,,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137276,137276,137276,223526,DLH2EN,NaN,3c6610,D-AIPP,A320,EDDF,EGBB,...,Germany,150.0,EDDF-EGBB-DLH,Normal,Normal,Normal,27,"WAW-BHX-leisure-ID39,WAW-BHX-leisure-ID47,WAW-...",12,12
137277,137277,137277,223529,EZY72RF,NaN,400a69,G-EZEB,A319,LFBO,EGKK,...,United Kingdom,124.0,LFBO-EGKK-EZY,Normal,Normal,Normal,22,"TLS-LGW-group-ID4,TLS-ARN-business-ID5,TLS-JFK...",9,9
137278,137278,137278,223532,NAX2PC,NaN,47947c,LN-NIH,B738,ENGM,ESSA,...,Norway,162.0,ENGM-ESSA-NAX,Normal,Normal,Normal,5,"OSL-LHR-business-ID3,OSL-LHR-leisure-ID196",2,2
137279,137279,137279,223538,BAW89ZV,NaN,406b5b,G-GATL,A320,LIPZ,EGKK,...,United Kingdom,150.0,LIPZ-EGKK-BAW,Normal,Normal,Normal,8,"VCE-LGW-business-ID13,VCE-DUB-group-ID4,VCE-DU...",3,3


In [39]:
combined_schedule

,Unnamed: 0,flight_id,callsign,number,icao24,registration,typecode,origin,destination,firstseen,...,Country_A,capacity,routeOperator,origin_outlier,destination_outlier,LegAirport_ICAO_outlier,occupancy,bookings,outPNRs,retPNRs
8,8,56,NAX7UA,DY7097,479089,LN-LNI,B789,LFPG,KLAX,2019-04-30 13:54:09+00:00,...,Norway,296.000000,LFPG-KLAX-NAX,Normal,Normal,Normal,4,"CDG-LAX-group-ID3,CDG-LAX-leisure-ID13",2,2
14,14,73,VIR23J,NaN,4073ce,NaN,NaN,EGLL,KLAX,2019-04-30 14:47:15+00:00,...,United Kingdom,304.328767,EGLL-KLAX-VIR,Normal,Normal,Normal,4,"LHR-LAX-leisure-ID3,LHR-LAX-leisure-ID29,LHR-L...",3,3
18,18,82,BAW44N,BA273,4065de,G-STBF,B77W,EGLL,KSAN,2019-04-30 15:02:48+00:00,...,United Kingdom,212.000000,EGLL-KSAN-BAW,Normal,Normal,Normal,1,LHR-SAN-leisure-ID2,1,1
21,21,91,AFR380V,NaN,3949f0,F-GSPQ,B772,SAEZ,LFPG,2019-04-30 15:20:17+00:00,...,France,312.000000,SAEZ-LFPG-AFR,Normal,Normal,Normal,2,CDG-EZE-leisure-ID75,0,1
34,34,118,ANZ1,NaN,c81d7e,ZK-OKM,B77W,EGLL,KHHR,2019-04-30 15:46:11+00:00,...,New Zealand,212.000000,EGLL-KHHR-ANZ,Normal,Normal,Normal,4,"LHR-HHR-leisure-ID61,LHR-HHR-leisure-ID24",2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629218,629218,1132052,RYR1YD,NaN,4cacdf,NaN,NaN,EIDW,EGMC,2019-04-30 21:47:04+00:00,...,Ireland,162.000000,EIDW-EGMC-RYR,Normal,Normal,Normal,11,"DXB-SEN-business-ID15,CDG-SEN-leisure-ID99,DXB...",4,4
629220,629220,1132062,EZY73YR,NaN,405b66,G-EZDM,A319,LJLJ,EGSS,2019-04-30 21:57:32+00:00,...,United Kingdom,124.000000,LJLJ-EGSS-EZY,Normal,Normal,Normal,1,CDG-LJU-leisure-ID15,0,1
629224,629224,1132074,FDX5233,NaN,a8aaf4,N658FE,A306,LIMC,LFPG,2019-04-30 22:08:16+00:00,...,United States,266.000000,LIMC-LFPG-FDX,Normal,Normal,Normal,4,"CDG-MXP-leisure-ID46,CDG-MXP-leisure-ID80",0,2
629227,629227,1132083,TAY826H,NaN,440575,OE-IAP,B734,LFPG,EBLG,2019-04-30 22:16:30+00:00,...,Belgium,146.000000,LFPG-EBLG-TAY,Normal,Normal,Normal,13,"LHR-LGG-leisure-ID44,CDG-LGG-leisure-ID17,DXB-...",6,6


In [47]:
df_book['no_flights']= df_book['allFlights'].apply(len)
df_book_cln = df_book[df_book['no_flights']!=0]
df_book_cln.shape

(1409093, 37)

In [64]:
df_P.columns

Index(['Unnamed: 0', 'P_num', 'HHID', 'P_ID', 'P_AGE', 'AgeRange', 'AgeGroup',
       'P_GENDER', 'GenderHOH', 'AgeHOH', 'SizeHH', 'HH_ISO', 'HHType', 'Lang',
       'Surname', 'Address', 'PostCode', 'Country', 'FirstName', 'DOB',
       'FreeEmail', 'PaymentInfo_VendorCode', 'PaymentInfo_ExpiryDate',
       'PaymentInfo_AccountNbr', 'WorkEmail', 'DOCS_ExpiryDate',
       'DOC_FirstName', 'DOC_Surname', 'TYP_FirstName', 'TYP_Surname',
       'NationalityNat'],
      dtype='object')

In [66]:
flattened_unique_list = list(set(item for sublist in df_book_cln['list_of_passengers'] for item in sublist))
flattened_unique_list
df_P_cln = df_P[df_P['P_ID'].isin(flattened_unique_list)]
df_P_cln


,Unnamed: 0,P_num,HHID,P_ID,P_AGE,AgeRange,AgeGroup,P_GENDER,GenderHOH,AgeHOH,...,PaymentInfo_VendorCode,PaymentInfo_ExpiryDate,PaymentInfo_AccountNbr,WorkEmail,DOCS_ExpiryDate,DOC_FirstName,DOC_Surname,TYP_FirstName,TYP_Surname,NationalityNat
0,0,1,ALB_0,ALB_0_1,43,40-44,AGE9,F,F,43,...,VISA 19 digit,2022-08-31,4604396803820495590,amyrichardson@tapia.biz,2024-02-14,Amy,Adams,Amy,Adams,ALB
1,1,2,ALB_0,ALB_0_2,86,85-89,AGE18,F,F,43,...,JCB 15 digit,2023-10-17,569834953618,jimenezkeith@jackson.com,2022-01-28,Margaret,Adams,Margaret,Adams,ALB
2,2,3,ALB_1,ALB_1_1,41,40-44,AGE9,M,M,41,...,VISA 19 digit,2023-07-17,4606996421384305,danielgonzalez@carey.com,2023-05-21,Micheal,Wells,Micheal,Wells,ALB
3,3,4,ALB_1,ALB_1_2,41,40-44,AGE9,F,M,41,...,Maestro,2023-11-23,4212297588736775,jennifer19@grant-hill.com,2025-03-29,Anne,Wells,Anne,Wells,ALB
7,7,8,ALB_3,ALB_3_1,71,70-74,AGE15,M,M,71,...,JCB 16 digit,2022-10-31,3514556265556393,xbailey@hooper.com,2024-06-01,Michael,Freudenthaler,Michael,Freudenthaler,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7753640,451784,451785,YEM_12741,YEM_12741_3,3,0-4,AGE1,F,F,40,...,-,-,-,-,2024-06-17,blnd,al bn Zfr,blnd,al bn Zfr,YEM
7753645,451789,451790,YEM_12744,YEM_12744_1,54,50-54,AGE11,M,M,54,...,VISA 16 digit,2022-09-21,6542499426719585,al-mqtkf@al.com,2023-08-19,mjd,al Hsyn,mjd,al Hsyn,YEM
7753646,451790,451791,YEM_12744,YEM_12744_2,49,45-49,AGE10,F,M,54,...,Diners Club / Carte Blanche,2023-04-08,3521529841782925,jwdybn-ldn@lhjr-mhn.com,2022-08-10,khsh`@,al Hsyn,khsh`@,al Hsyn,YEM
7753647,451791,451792,YEM_12744,YEM_12744_3,28,25-29,AGE6,F,M,54,...,Maestro,2022-04-28,4540019301249831829,ljbrsm@al.com,2024-07-29,'Gryd,al Hsyn,'Gryd,al Hsyn,YEM


In [67]:
# combined_schedule.to_csv('data/buildXML2/DB/flights.csv')
df_P_cln.to_csv('data/buildXML2/DB/passenger.csv')
# df_book_cln.to_csv('data/buildXML2/DB/bookings.csv')

Index(['Unnamed: 0', 'Unnamed: 0.1', 'initID', 'list_of_passengers', 'route',
       'num_in_party', 'BookingAgency', 'BookingDay', 'Unnamed: 0.1.1',
       'IATA_O', 'IATA_D', 'HH_ISO_D', 'region_D', 'HH_ISO_O', 'region_O',
       'LegHH_ISO', 'LegRegion', 'Origin', 'Destination', 'fixRoute',
       'Leg_fixRoute', 'stayDay', 'Airport_Itinerary_Out', 'full_routes',
       'outboundDate', 'returnDate', 'outboundFlights',
       'Airport_Itinerary_Ret', 'return_routes', 'returnFlights', 'IATA',
       'Currency', 'HH_ISO', 'City', 'primary_passengers', 'allFlights',
       'no_flights'],
      dtype='object')